In [1]:
!pip install grpcio==1.42.0 tensorflow-serving-api==2.7.0
!pip install keras-image-helper

import grpc
import tensorflow as tf
from tensorflow_serving.apis import predict_pb2
from tensorflow_serving.apis import prediction_service_pb2_grpc

host = 'localhost:8500'
channel = grpc.insecure_channel(host)
stub = prediction_service_pb2_grpc.PredictionServiceStub(channel)

  Using cached grpcio-1.42.0-cp39-cp39-macosx_10_10_x86_64.whl (4.0 MB)
  Using cached tensorboard-2.15.0-py3-none-any.whl (5.6 MB)
INFO: pip is looking at multiple versions of opt-einsum to determine which version is compatible with other requirements. This could take a while.
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
INFO: pip is looking at multiple versions of numpy to determine which version is compatible with other requirements. This could take a while.
  Using cached numpy-1.26.2-cp39-cp39-macosx_10_9_x86_64.whl (20.6 MB)
INFO: pip is looking at multiple versions of ml-dtypes to determine which version is compatible with other requirements. This could take a while.
  Using cached ml_dtypes-0.2.0-cp39-cp39-macosx_10_9_universal2.whl (1.2 MB)
INFO: pip is looking at multiple versions of libclang to determine which version is compatible with other requirements. This could take a while.
  Using cached libclang-16.0.6-py2.py3-none-macosx_10_9_x86_64.whl (24.5 MB)
INFO: 

  Using cached tensorflow-2.15.0-cp39-cp39-macosx_10_15_x86_64.whl (239.1 MB)
  Using cached tensorflow-2.14.1-cp39-cp39-macosx_10_15_x86_64.whl (229.7 MB)
  Using cached keras-2.14.0-py3-none-any.whl (1.7 MB)
  Using cached tensorflow_estimator-2.14.0-py2.py3-none-any.whl (440 kB)
  Using cached tensorboard-2.14.1-py3-none-any.whl (5.5 MB)
  Using cached tensorboard-2.14.0-py3-none-any.whl (5.5 MB)
  Using cached tensorflow-2.14.0-cp39-cp39-macosx_10_15_x86_64.whl (229.6 MB)
  Using cached tensorflow-2.13.1-cp39-cp39-macosx_10_15_x86_64.whl (216.2 MB)
  Using cached numpy-1.24.3-cp39-cp39-macosx_10_9_x86_64.whl (19.8 MB)
  Using cached keras-2.13.1-py3-none-any.whl (1.7 MB)
  Using cached tensorboard-2.13.0-py3-none-any.whl (5.6 MB)
  Using cached tensorflow_estimator-2.13.0-py2.py3-none-any.whl (440 kB)
  Using cached tensorflow-2.13.0-cp39-cp39-macosx_10_15_x86_64.whl (216.2 MB)
  Using cached tensorflow-2.12.1-cp39-cp39-macosx_10_15_x86_64.whl (230.1 MB)
  Using cached keras-2.12.0

2023-12-10 00:25:05.143070: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/Users/muratkahraman/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
from keras_image_helper import create_preprocessor

preprocessor = create_preprocessor('xception', target_size=(299, 299))
url='https://images.theconversation.com/files/513157/original/file-20230302-28-r91z9l.jpg?ixlib=rb-1.1.0&rect=10%2C0%2C6699%2C4466&q=45&auto=format&w=926&fit=clip'
X = preprocessor.from_url(url)

In [4]:
def np_to_protobuf(data):
    return tf.make_tensor_proto(data, shape=data.shape)

pb_request = predict_pb2.PredictRequest()
pb_request.model_spec.name = 'jelly-model'
pb_request.model_spec.signature_name = 'serving_default'
pb_request.inputs['input_31'].CopyFrom(np_to_protobuf(X))

In [7]:
pb_response = stub.Predict(pb_request, timeout=20.0)

preds=pb_response.outputs['dense_23'].float_val

In [8]:
classes=['Moon_jellyfish',
 'barrel_jellyfish',
 'blue_jellyfish',
 'compass_jellyfish',
 'lions_mane_jellyfish',
 'mauve_stinger_jellyfish']

In [9]:
dict(zip(classes, preds))

{'Moon_jellyfish': 0.14447933435440063,
 'barrel_jellyfish': 0.24286234378814697,
 'blue_jellyfish': 2.6390597820281982,
 'compass_jellyfish': -0.09297686815261841,
 'lions_mane_jellyfish': 0.03131067752838135,
 'mauve_stinger_jellyfish': 0.9337836503982544}

In [10]:
print(tf.__version__)

2.11.1
